In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.1. Data Imputation')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.1. Data Imputation/Output')
cur_date = "080823"

library(missForest)
library(imputeLCMD)
library(readxl)
library(openxlsx)
library(tidyverse)

# reading in file
well_df = data.frame(read_excel("Input/Union_WellData_080723.xlsx", sheet = 2))

# converting some cols to numeric
numeric_df = data.frame(apply(well_df[,c(5,7,9,11,13,14,19)], 2, as.numeric))
# converting one col to charcter
well_df$Landuse = as.character(well_df$Landuse)
recombined_df = cbind(well_df[,-c(5,7,9,11,13,14,19)], numeric_df)
well_df = recombined_df[,c(1:4,13,5,14,6,15,9:11,17:19,12,7,16,8)]

Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix

Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
and has been superseded by the norm2 package.

Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownam

In [2]:
head(well_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Belt_Geocode,Map_Unit_Symbol,Landuse,Longtitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,41176,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Ar,NA,ND
2,1006004,960,12-155,41176,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Mn,440,D
3,1006004,960,12-155,41176,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Cr,NA,ND
4,1024009,1803,15-177,42355,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Ar,NA,ND
5,1024009,1803,15-177,42355,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Mn,690,D
6,1024009,1803,15-177,42355,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Cr,NA,ND


Imputing missing data. Random forest will be used to input casing depth, well depth, static water depth, flow rate, and pH. Quantile regression imputation of left-censored data (QRILC) will be used to impute each metal concentration.

Typically background filters would be implemented for all data, but only the variable `Concentration` is missing a significant portion of data.

First, we're only keeping data that has a latitude and longtitude, since quite a few of the variables have a significant amount of missing data.

In [3]:
smaller_well_df = well_df[!is.na(well_df$Longtitude),]
head(smaller_well_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Belt_Geocode,Map_Unit_Symbol,Landuse,Longtitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,41176,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Ar,NA,ND
2,1006004,960,12-155,41176,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Mn,440,D
3,1006004,960,12-155,41176,52,165,41,60,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Cr,NA,ND
4,1024009,1803,15-177,42355,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Ar,NA,ND
5,1024009,1803,15-177,42355,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Mn,690,D
6,1024009,1803,15-177,42355,40,445,42,2,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Cr,NA,ND


In [4]:
# splitting the dataset based on the metal
preimputed_df = smaller_well_df %>%
    group_by(Metal) %>%
    group_split()

arsenic_df = preimputed_df[[1]]
chromium_df = preimputed_df[[2]]
manganese_df = preimputed_df[[3]]

# RF Imputation

In [5]:
#imputing cols 5-9 and 15 using RF
random_forest_imputation = function(dataset){
    
    imputed_RF_object = missForest(as.matrix(dataset[,c(5:9,16)]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(dataset[,1:4], imputed_RF_df[,1:5], dataset[,10:15], imputed_RF_df[,6],
                                       dataset[,17:19])) 
    
    #idk why i have to do this
    colnames(imputed_final_df)[16] = c("Elevation")
    
    return(imputed_final_df)
}

# calling fn
rf_imputed_arsenic_df = random_forest_imputation(arsenic_df)
rf_imputed_chromium_df = random_forest_imputation(chromium_df)
rf_imputed_manganese_df = random_forest_imputation(manganese_df)

head(rf_imputed_arsenic_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Belt_Geocode,Map_Unit_Symbol,Landuse,Longtitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,41176,52,165,41,60.0,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Ar,NA,ND
2,1024009,1803,15-177,42355,40,445,42,2.0,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Ar,NA,ND
3,1054019,1647,14-212,42037,45,160,40,40.0,7.4,Carolina Slate Belt - CZmd3,GsB,81,-80.33170,35.16158,634.2669,692.789,Ar,NA,ND
4,1057017,970,12-169,41204,42,440,57,1.5,8.0,Carolina Slate Belt - CZmd2,BaB,42,-80.32263,35.13962,855.4713,692.789,Ar,35,D
5,1060006,533,10-239,40546,48,120,42,25.0,7.1,Carolina Slate Belt - CZmd2,BdB2,43,-80.32911,35.13027,163.5688,683.387,Ar,NA,ND
6,1066006,1795,15-224,42353,60,280,32,10.0,8.2,Carolina Slate Belt - CZmd2,BaB,22,-80.32205,35.10479,725.7672,614.599,Ar,53,D


# QRILC Imputation

In [6]:
# QRILC
QRILC_imputation = function(dataset){
    # """
    # Creating a function to generate missing data
    # :param (input): df with missing data
    # :output: 1 imputed df
    # """
  
    # normalizing data since that's what the QRILC function wants
    # ended up imputing another col (health dept id), since this function needs > 1 col to impute
    QRILC_prep = dataset[,c(2,18)] %>%
         mutate_all(., function(x) log2(x)) %>%
         as.matrix()

    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log2_df = data.frame(imputed_QRILC_object[1]) 
    
    # converting back the original scale
    QRILC_df = QRILC_log2_df %>%
        mutate_all(., function(x) 2^x)
    
    # adding back in other data
    QRILC_df = data.frame(cbind(dataset[,1:17], QRILC_df[,2], dataset[,19]))
    colnames(QRILC_df)[18:19] = c("Concentration", "Detect_Concentration")


   return(QRILC_df)
}

# calling fn
imputed_arsenic_df = QRILC_imputation(rf_imputed_arsenic_df)
imputed_chromium_df = QRILC_imputation(rf_imputed_chromium_df)
imputed_manganese_df = QRILC_imputation(rf_imputed_manganese_df)

In [7]:
# combining in 1 df
imputed_df = rbind(imputed_arsenic_df, imputed_chromium_df, imputed_manganese_df)
head(imputed_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Belt_Geocode,Map_Unit_Symbol,Landuse,Longtitude,Latitude,Stream_Distance,Elevation,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,41176,52,165,41,60.0,7.7,Carolina Slate Belt - CZmd3,BdB2,81,-80.29918,35.17970,811.9613,611.499,Ar,2.473682,ND
2,1024009,1803,15-177,42355,40,445,42,2.0,7.3,Carolina Slate Belt - CZfv2,GsB,81,-80.31061,35.15487,341.7654,688.452,Ar,1.863728,ND
3,1054019,1647,14-212,42037,45,160,40,40.0,7.4,Carolina Slate Belt - CZmd3,GsB,81,-80.33170,35.16158,634.2669,692.789,Ar,1.363029,ND
4,1057017,970,12-169,41204,42,440,57,1.5,8.0,Carolina Slate Belt - CZmd2,BaB,42,-80.32263,35.13962,855.4713,692.789,Ar,35.000000,D
5,1060006,533,10-239,40546,48,120,42,25.0,7.1,Carolina Slate Belt - CZmd2,BdB2,43,-80.32911,35.13027,163.5688,683.387,Ar,1.321271,ND
6,1066006,1795,15-224,42353,60,280,32,10.0,8.2,Carolina Slate Belt - CZmd2,BaB,22,-80.32205,35.10479,725.7672,614.599,Ar,53.000000,D


In [8]:
# exporting results
write.xlsx(imputed_df, paste0(Output,"/", "Imputed_Well_Data_", cur_date, ".xlsx"), rowNames = FALSE)